In [ ]:
# v2: 해상도 높이기

In [ ]:
# v3: 마커 여러 개 인식하고 id와 면적 (픽셀) 출력하기

In [ ]:
# v4: aruco pose estimation

In [4]:
import cv2
import cv2.aruco as aruco
import numpy as np

# 🎯 카메라 캘리브레이션 값 로드 (미리 보정된 값)
calibration_matrix_path = "C:/calibration_matrix.npy"
distortion_coefficients_path = "C:/distortion_coefficients.npy"

k = np.load(calibration_matrix_path)  # 카메라 행렬
d = np.load(distortion_coefficients_path)  # 왜곡 계수
print("✅ 카메라 행렬:\n", k)
print("✅ 왜곡 계수:\n", d)

# 🎯 카메라 열기
cap = cv2.VideoCapture(1)  # 필요 시 1로 변경
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# 🎯 ArUco 마커 딕셔너리 설정 (4x4, 50개)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
aruco_params = aruco.DetectorParameters()
detector = aruco.ArucoDetector(aruco_dict, aruco_params)

# 🎯 ArUco 마커 크기 (미터 단위, 0.05m = 5cm)
MARKER_SIZE = 0.05  # 5cm

def find_aruco_markers(img, draw=True):
    """ ArUco 마커 감지 및 Pose Estimation 수행 """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = detector.detectMarkers(gray)

    marker_info = []

    if ids is not None:
        for i, corner in enumerate(corners):
            marker_id = ids[i][0]

            # 🎯 마커의 2D 좌표 가져오기
            x1, y1 = corner[0][0]
            x2, y2 = corner[0][1]
            x3, y3 = corner[0][2]
            x4, y4 = corner[0][3]

            # 🎯 마커 면적 계산
            area = 0.5 * abs((x1*y2 + x2*y3 + x3*y4 + x4*y1) - (y1*x2 + y2*x3 + y3*x4 + y4*x1))

            # 🎯 마커의 3D 위치(Pose) 계산
            rvec, tvec, _ = aruco.estimatePoseSingleMarkers(corner, MARKER_SIZE, k, d)

            # 🎯 화면에 마커 정보 표시
            if draw:
                aruco.drawDetectedMarkers(img, [corner])
                cv2.putText(img, f"ID: {marker_id}, Area: {int(area)} px^2",
                            (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # 🎯 3D 좌표 축 표시 (빨강, 초록, 파랑)
                cv2.drawFrameAxes(img, k, d, rvec, tvec, 0.03)

            # 🎯 검출된 마커 정보 저장
            marker_info.append((marker_id, area, tvec[0][0], rvec[0][0]))

    return marker_info

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ 카메라 프레임을 읽을 수 없습니다.")
        break

    markers = find_aruco_markers(frame)

    for marker in markers:
        marker_id, area, tvec, rvec = marker
        print(f"🔹 마커 ID: {marker_id}")
        print(f"   📏 면적: {int(area)} px^2")
        print(f"   📍 3D 위치: X={tvec[0]:.3f}m, Y={tvec[1]:.3f}m, Z={tvec[2]:.3f}m")
        print(f"   🔄 회전 벡터: Rx={rvec[0]:.3f}, Ry={rvec[1]:.3f}, Rz={rvec[2]:.3f}")

    # 🎯 화면 출력
    cv2.imshow("ArUco Detection", frame)

    # 'q' 또는 ESC 키로 종료
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:
        break

cap.release()
cv2.destroyAllWindows()


✅ 카메라 행렬:
 [[992.40913116   0.         605.19256803]
 [  0.         996.28675761 343.67586117]
 [  0.           0.           1.        ]]
✅ 왜곡 계수:
 [[ 0.02206642  0.20438685 -0.00633739 -0.00140045 -0.85132748]]
🔹 마커 ID: 0
   📏 면적: 36942 px^2
   📍 3D 위치: X=-0.046m, Y=0.071m, Z=0.258m
   🔄 회전 벡터: Rx=3.101, Ry=0.214, Rz=0.549
🔹 마커 ID: 0
   📏 면적: 37665 px^2
   📍 3D 위치: X=-0.044m, Y=0.065m, Z=0.257m
   🔄 회전 벡터: Rx=2.639, Ry=0.048, Rz=0.044
🔹 마커 ID: 0
   📏 면적: 38025 px^2
   📍 3D 위치: X=-0.042m, Y=0.061m, Z=0.257m
   🔄 회전 벡터: Rx=2.695, Ry=0.042, Rz=0.039
🔹 마커 ID: 0
   📏 면적: 38706 px^2
   📍 3D 위치: X=-0.040m, Y=0.056m, Z=0.257m
   🔄 회전 벡터: Rx=2.873, Ry=0.118, Rz=0.327
🔹 마커 ID: 0
   📏 면적: 38407 px^2
   📍 3D 위치: X=-0.040m, Y=0.052m, Z=0.257m
   🔄 회전 벡터: Rx=2.837, Ry=0.085, Rz=0.142
🔹 마커 ID: 0
   📏 면적: 37939 px^2
   📍 3D 위치: X=-0.039m, Y=0.048m, Z=0.258m
   🔄 회전 벡터: Rx=2.825, Ry=0.090, Rz=0.153
🔹 마커 ID: 0
   📏 면적: 38150 px^2
   📍 3D 위치: X=-0.037m, Y=0.045m, Z=0.256m
   🔄 회전 벡터: Rx=2.759, Ry=0.069,